Obrir tocho

In [1]:
#Let's import everything!
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
#Read files into a pd dataframe
#Training file from a random selection of the original large file to be able to work faster
training = pd.read_csv('tocho.csv', index_col = 0)

In [3]:
#update dataset with deleted tweets and users ids (Still can't find a quick and efficient way to do it)
'''
with open('deleted_user_IDs.txt') as f:
    deleted_user_ID = f.read().splitlines()
with open('deleted_tweet_IDs.txt') as f:
    deleted_tweet_ID = f.read().splitlines()
    
training['todelete'] = training['user'].apply(lambda x: x in eleted_user_ID)
training['todelete2'] = training['tweet_id'].apply(lambda x: x in deleted_tweet_ID)

'''

"\nwith open('deleted_user_IDs.txt') as f:\n    deleted_user_ID = f.read().splitlines()\nwith open('deleted_tweet_IDs.txt') as f:\n    deleted_tweet_ID = f.read().splitlines()\n    \ntraining['todelete'] = training['user'].apply(lambda x: x in eleted_user_ID)\ntraining['todelete2'] = training['tweet_id'].apply(lambda x: x in deleted_tweet_ID)\n\n"

In [4]:
#Let's explore the columns first
training.head(5)

,text_ tokens,hashtags,tweet_id,present_media,present_links,present_domains,tweet_type,language,tweet_timestamp,engaged_with_user_id,...,engaging_user_id,engaging_user_follower_count,engaging_user_following_count,engaging_user_is_verified,engaging_user_account_creation,engagee_follows_engager,reply_timestamp,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp
0,101\t16493\t12478\t117\t10105\t42370\t76299\t1...,NaN,39024FBE0136E046D1357196BAECFCA6,GIF,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581532200,3BF0702AA3337E076DA5DF54E55117D1,...,000006829BEADA9EEA695CF0C334B426,2,29,False,1568107028,False,NaN,NaN,NaN,NaN
1,101\t11420\t10173\t45283\t10133\t40960\t10835\...,NaN,6F4B64CF616A1B65C94B9581AB3617E5,NaN,NaN,NaN,TopLevel,ECED8A16BE2A5E8871FD55F4842F16B1,1581134453,4C86333F8DC95B0D3C716C46A7862CCC,...,00046D54CBEE1354F395930AB602C6EF,87,100,False,1500643248,False,NaN,NaN,NaN,1.581158e+09
2,101\t10105\t76456\t22201\t119\t119\t119\t119\t...,NaN,6DD591ABD65945B412C062DE26C55CAC,NaN,NaN,NaN,Quote,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581340818,304AD9FCC24B58A12FC7843F8DFE804E,...,000AF284D93A0EBDD1BE0D4701F581FC,583,1071,False,1465415499,True,NaN,NaN,NaN,1.581345e+09
3,101\t49307\t11090\t49339\t11369\t14703\t110014...,B929FC51754F08E9CF07498FF4DFC1DC,25BFBE285D27B2144C6B169C11083FAB,Photo,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581332785,AF003AF390475BFCB3D3618FE55F950E,...,001C14131FC5556F3CB6555FF04F0AB1,272,1874,False,1425137677,False,NaN,NaN,NaN,1.581344e+09
4,101\t52212\t18010\t10120\t15122\t10325\t40323\...,NaN,27A0AC9D2924771C97F7082EE951C1B1,Photo,NaN,NaN,TopLevel,717293301FE296B0B61950D041485825,1581108476,1699A4DEBFEAB17CC8F748C13F889C35,...,001DD4110BC58DBDC235D0B7A5B25ACE,1272,1243,False,1305546726,False,NaN,NaN,NaN,1.581109e+09


In [5]:
#For the tokens, let's split them into a list
training['text_ tokens'] = training['text_ tokens'].str.split('\t')

Let's explore all the columns a bit more one by one

In [6]:
training['present_media'].value_counts(dropna=False)

NaN                           282152
Photo                          84066
Video                          49280
Photo\tPhoto                   18475
Photo\tPhoto\tPhoto\tPhoto     10528
Photo\tPhoto\tPhoto             5834
GIF                             4947
Video\tVideo                       3
Photo\tVideo                       1
Video\tVideo\tVideo\tVideo         1
Name: present_media, dtype: int64

In [7]:
#First, we will split them into a list
training['present_media'] = training['present_media'].str.split('\t')

In [8]:
#Now, let's do a one hot encoding
def hot_encod_photo(x):
    try:
        return x.count('Photo')
    except AttributeError:
        return 0
def hot_encod_video(x):
    try:
        return x.count('Video')
    except AttributeError:
        return 0
def hot_encod_gif(x):
    try:
        return x.count('GIF')
    except AttributeError:
        return 0

training['photo'] = training['present_media'].apply(hot_encod_photo)
training['video'] = training['present_media'].apply(hot_encod_video)
training['gif'] = training['present_media'].apply(hot_encod_gif)

training.drop(['present_media'], axis=1, inplace=True)

In [9]:
training['present_links'].value_counts(dropna=False)

NaN                                 391258
E522B6C629B01DA6B29DE65BF5DFA55B       126
DEA673FD63DDDC8A3606106D5CA235B0        50
79F5D3D2AECF3DE531946BABEF8BD0E7        44
6DDBD9DAAA2AE38B8EBB54BA8E7098DD        44
                                     ...  
92BC61ACA6636B156F83B04B9EAC7DEA         1
C138104740ED9C7AE866CA2DCB7092A2         1
68B55836AFA0477EF3420157404EB99E         1
1CF9315AB6FA6C793F6EF57425A744C9         1
8EF94EB9E7695B9595E2B4BC328728A4         1
Name: present_links, Length: 55426, dtype: int64

What should we do in this case? We don't want to drop de Nans, we could so:
* Put a false link that will represent an absence of a link
* generate random codes for false links?

By now, we will keep it without changes, we still have to think how to proceed

In [10]:
training['tweet_type'].value_counts(dropna=False)

TopLevel    261283
Retweet     154015
Quote        39989
Name: tweet_type, dtype: int64

We'll do the same than with the present_media case'

In [11]:
training['categorical_type'] = pd.Categorical(training['tweet_type'])
dfDummies = pd.get_dummies(training['categorical_type'], prefix = 'engagement_type')
training=pd.concat([training, dfDummies], axis=1)
training.drop(['categorical_type','tweet_type'], axis=1,inplace=True)

We will deal with the timestamps now!

In [13]:
training.head(5)

,text_ tokens,hashtags,tweet_id,present_links,present_domains,language,tweet_timestamp,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,...,retweet_timestamp,retweet_with_comment_timestamp,like_timestamp,photo,video,gif,engagement_type_Quote,engagement_type_Retweet,engagement_type_TopLevel,engagement_timestamp
0,"[101, 16493, 12478, 117, 10105, 42370, 76299, ...",NaN,39024FBE0136E046D1357196BAECFCA6,NaN,NaN,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581532200,3BF0702AA3337E076DA5DF54E55117D1,1344732,125,...,NaN,NaN,NaN,0,0,1,0,0,1,NaN
1,"[101, 11420, 10173, 45283, 10133, 40960, 10835...",NaN,6F4B64CF616A1B65C94B9581AB3617E5,NaN,NaN,ECED8A16BE2A5E8871FD55F4842F16B1,1581134453,4C86333F8DC95B0D3C716C46A7862CCC,571561,283,...,NaN,NaN,1.581158e+09,0,0,0,0,0,1,1.581158e+09
2,"[101, 10105, 76456, 22201, 119, 119, 119, 119,...",NaN,6DD591ABD65945B412C062DE26C55CAC,NaN,NaN,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581340818,304AD9FCC24B58A12FC7843F8DFE804E,9533,421,...,NaN,NaN,1.581345e+09,0,0,0,1,0,0,1.581345e+09
3,"[101, 49307, 11090, 49339, 11369, 14703, 11001...",B929FC51754F08E9CF07498FF4DFC1DC,25BFBE285D27B2144C6B169C11083FAB,NaN,NaN,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581332785,AF003AF390475BFCB3D3618FE55F950E,60347,110,...,NaN,NaN,1.581344e+09,1,0,0,0,0,1,1.581344e+09
4,"[101, 52212, 18010, 10120, 15122, 10325, 40323...",NaN,27A0AC9D2924771C97F7082EE951C1B1,NaN,NaN,717293301FE296B0B61950D041485825,1581108476,1699A4DEBFEAB17CC8F748C13F889C35,376180,21,...,NaN,NaN,1.581109e+09,1,0,0,0,0,1,1.581109e+09


In [15]:
# If we look at the timestamps we have 3 columns (sometimes values in more tha one or other times NaNs), 
# for now let's only keep the first one of them, and create a new column to see wich are the false positives
training['engagement_timestamp'] = training[['reply_timestamp','retweet_with_comment_timestamp','like_timestamp']].min(axis=1)
training.drop(['reply_timestamp','retweet_with_comment_timestamp','like_timestamp'], axis=1, inplace=True)

# And let's create a new column with a 0 when there iss no engagement, and a 1 when it exists
training['engagement_exists']=training['engagement_timestamp'].apply(lambda x: 0 if (np.isnan(x)) else 1)

In [16]:
training.head(3)

,text_ tokens,hashtags,tweet_id,present_links,present_domains,language,tweet_timestamp,engaged_with_user_id,engaged_with_user_follower_count,engaged_with_user_following_count,...,engagee_follows_engager,retweet_timestamp,photo,video,gif,engagement_type_Quote,engagement_type_Retweet,engagement_type_TopLevel,engagement_timestamp,engagement_exists
0,"[101, 16493, 12478, 117, 10105, 42370, 76299, ...",NaN,39024FBE0136E046D1357196BAECFCA6,NaN,NaN,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581532200,3BF0702AA3337E076DA5DF54E55117D1,1344732,125,...,False,NaN,0,0,1,0,0,1,NaN,0
1,"[101, 11420, 10173, 45283, 10133, 40960, 10835...",NaN,6F4B64CF616A1B65C94B9581AB3617E5,NaN,NaN,ECED8A16BE2A5E8871FD55F4842F16B1,1581134453,4C86333F8DC95B0D3C716C46A7862CCC,571561,283,...,False,NaN,0,0,0,0,0,1,1.581158e+09,1
2,"[101, 10105, 76456, 22201, 119, 119, 119, 119,...",NaN,6DD591ABD65945B412C062DE26C55CAC,NaN,NaN,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581340818,304AD9FCC24B58A12FC7843F8DFE804E,9533,421,...,True,NaN,0,0,0,1,0,0,1.581345e+09,1


In [17]:
training['engaging_user_is_verified'].value_counts(dropna=False)

False    454290
True        997
Name: engaging_user_is_verified, dtype: int64

In [18]:
training['engaged_with_user_is_verified'].value_counts(dropna=False)

False    335461
True     119826
Name: engaged_with_user_is_verified, dtype: int64

In [19]:
training['hashtags'].value_counts(dropna=False)

NaN                                                                                                                                       363830
5671B8B6E92C2A316C05A7CD2F17366C                                                                                                             974
18F6AC96A6EA62716A47DE9FE6241534                                                                                                             518
31C316EB7BB8CD1778CB83B8AA2603D9                                                                                                             275
B6024D6F02AFAF9BC8ED941F72C228F1                                                                                                             262
                                                                                                                                           ...  
52CAFD1FB3DD122609D956B09B2A3E44\tA41DA909E324886A615900DA9A155E61\tB7E24CD92E7329F9531ECC92A1543C99\tB2D2614B65CAF452BEA8C8925004

With that we will have to follow a similar strategy than with the present_links